# dotnet/interactive Github Dashboard with F# <img src ="https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Jupyter_logo.svg/207px-Jupyter_logo.svg.png" width="80px" alt="dotnet bot in space" align ="right"><img src ="https://user-images.githubusercontent.com/2546640/56708992-deee8780-66ec-11e9-9991-eb85abb1d10a.png" width="80px" alt="dotnet bot in space" align ="right">

### Add NuGet package references 

In [1]:
#r "nuget:Octokit, 0.32.0"
#r "nuget:NodaTime, 2.4.6"

open Octokit
open NodaTime
open NodaTime.Extensions
open XPlot.Plotly

Installed package Octokit version 0.32.0

Installed package NodaTime version 2.4.6

### Setup

Create a GitHub public API client

In [2]:
let organization = "dotnet"
let repositoryName = "interactive"
let options = ApiOptions()
let gitHubClient = GitHubClient(ProductHeaderValue("notebook"))

[Generate a user token](https://help.github.com/en/github/authenticating-to-github/creating-a-personal-access-token-for-the-command-line) to get rid of public [api](https://github.com/octokit/octokit.net/blob/master/docs/getting-started.md) throttling policies for anonymous users 

In [3]:
// let tokenAuth = Credentials("YOUR-TOKEN-HERE")
// gitHubClient.Credentials <- tokenAuth

In [4]:
let today = SystemClock.Instance.InUtc().GetCurrentDate()
let startOfTheMonth = today.With(DateAdjusters.Month(1))
let startOfTheYear = LocalDate(today.Year, 1, 1).AtMidnight()

let since t = Nullable(DateTimeOffset(t))

Query GitHub for : 
- Issues created this month
- Issues closed this month
- Every issue this year

In [5]:
let createdIssuesRequest =
    RepositoryIssueRequest(
        Since = since (startOfTheMonth.ToDateTimeUnspecified()),
        Filter = IssueFilter.Created)
let closedIssuesRequest =
    new RepositoryIssueRequest(
                Since = since (startOfTheMonth.ToDateTimeUnspecified()),
                State = ItemStateFilter.Closed)
let thisYearIssuesRequest =
    RepositoryIssueRequest(
        Since = since (startOfTheYear.ToDateTimeUnspecified()),
        State = ItemStateFilter.All)

Start pulling data via the GitHub API

In [6]:
let createdThisMonthTask =
    async {
        return!
            gitHubClient.Issue.GetAllForRepository(organization, repositoryName, createdIssuesRequest)
            |> Async.AwaitTask
    }

let closedThisMonthTask =
    async {
        return!
            gitHubClient.Issue.GetAllForRepository(organization, repositoryName, closedIssuesRequest)
            |> Async.AwaitTask
    }

let thisYearIssuesTask =
    async {
        return!
            gitHubClient.Issue.GetAllForRepository(organization, repositoryName, thisYearIssuesRequest)
            |> Async.AwaitTask
    }

let results =
    [| createdThisMonthTask; closedThisMonthTask; thisYearIssuesTask |]
    |> Async.Parallel
    |> Async.RunSynchronously

let createdThisMonth = results.[0]
let closedThisMonth = results.[1]
let thisYearIssues = results.[2]

Group open and closed issues by month

In [7]:
let openSoFar =
    createdThisMonth
    |> Seq.sortBy (fun i -> i.CreatedAt)
    |> Seq.filter (fun i -> i.State.StringValue = "open")

let openByMonthOfCreation =
    openSoFar
    |> Seq.groupBy (fun i -> {| Year = i.CreatedAt.Year; Month = i.CreatedAt.Month |})
    |> Seq.map (fun (key, issues) -> {| Date = key; Count = issues.Count() |})
    
let closedSoFar =
    thisYearIssues
    |> Seq.sortBy (fun i -> i.CreatedAt)
    |> Seq.filter (fun i -> i.State.StringValue = "closed")

let closedByMonthOfClosure =
    closedSoFar
    |> Seq.groupBy (fun i -> {| Year = i.ClosedAt.Value.Year; Month = i.ClosedAt.Value.Month |})
    |> Seq.map (fun (key, issues) ->  {| Date = key; Count = issues.Count() |})

let openCountByMonth =
    let mutable runningTotal = thisYearIssues.Count
    
    closedSoFar
    |> List.ofSeq
    |> List.groupBy (fun i -> {| Year = i.CreatedAt.Year; Month = i.CreatedAt.Month |})
    |> List.map (fun (key, issues) ->
                   let dataPoint = {| Date = key; Count = issues.Count() |}
                   dataPoint)

Show issues opened this month grouped by day 

In [8]:
let createdThisMonthByDay =
    createdThisMonth
    |> Seq.groupBy (fun i -> DateTime(i.CreatedAt.Year,i.CreatedAt.Month, i.CreatedAt.Day))
    |> Seq.map (fun (date, issues) -> (date, issues.Count()))

createdThisMonthByDay
|> Chart.Line
|> Chart.WithTitle("Daily created issues over the past year")

Show open issues, in descending order. Limit to 10 to save screen space.

In [9]:
openSoFar
|> Seq.map (fun i -> {| CreatedAt = i.CreatedAt; Title = i.Title; State = i.State.StringValue; Number = i.Number |})
|> Seq.sortByDescending (fun d -> d.CreatedAt)
|> Seq.take 10 // Limiting the output to 10 here!

index,CreatedAt,Number,State,Title
0,2020-01-31 23:21:54Z,107,open,Support sending input_request and receiving input_reply messages via the stdin channel
1,2020-01-31 22:23:47Z,106,open,make kernel server honor `--log-path` and listen to kernel logging events
2,2020-01-31 16:16:46Z,105,open,Directives should be able to display help
3,2020-01-28 14:42:38Z,95,open,Execution failed if project references several other projects
4,2020-01-25 18:17:06Z,88,open,EntityFrameworkCore: System.BadImageFormatException: Could not load file or assembly 'Microsoft.Bcl.AsyncInterfaces
5,2020-01-18 13:00:04Z,83,open,Is there any optimization option when running c# codes?
6,2020-01-13 20:18:27Z,18,open,"After using #r ""nuget"" line numbers of errors do not line up correctly"
7,2020-01-13 12:49:35Z,17,open,Unable to use Tensors in Jupyter notebook due to loading errors
8,2019-12-22 22:51:10Z,15,open,Autocomplete Support
9,2019-12-19 21:41:33Z,14,open,CosmosClient in local Notebook throws exceptions


Let's see what issues still opened, grouped by month, looks like

In [10]:
openByMonthOfCreation
|> Seq.map (fun g -> (DateTime(g.Date.Year, g.Date.Month, 1), g.Count))
|> Chart.Line
|> Chart.WithTitle("Issues still opened, grouped by month")

Now let's look at idle vs active issues.

In [11]:
let idleByMonth =
    openSoFar
    |> Seq.filter (fun i -> i.Comments = 0)
    |> Seq.groupBy (fun i -> DateTime(i.CreatedAt.Year, i.CreatedAt.Month, 1))
    |> Seq.map(fun (key, issues) -> {| Date = key; Count = issues.Count() |})

let activeByMonth =
    openSoFar
    |> Seq.filter (fun i -> i.Comments > 0)
    |> Seq.groupBy (fun i -> DateTime(i.CreatedAt.Year, i.CreatedAt.Month, 1))
    |> Seq.map (fun (key, issues) -> {| Date = key; Count = issues.Count() |})

let idleSeries =
    Graph.Scattergl(
        name = "Idle",
        y = (idleByMonth |> Seq.map (fun g -> g.Count)),
        x = (idleByMonth |> Seq.map (fun g -> g.Date)))

let activeSeries =
    Graph.Scattergl(
        name = "Active",
        y = (activeByMonth |> Seq.map (fun g -> g.Count)),
        x = (activeByMonth |> Seq.map (fun g -> g.Date)))

[idleSeries; activeSeries]
|> Chart.Plot
|> Chart.WithTitle("Idle and active open issue report")

Now let's generate a report for the whole year.

In [12]:
let openDataPoints =
    openByMonthOfCreation
    |> Seq.map (fun g -> {| Date = DateTime(g.Date.Year, g.Date.Month, 1); Count = g.Count |})
    |> Seq.sortBy (fun d -> d.Date)

let closedDataPoints =
    closedByMonthOfClosure
    |> Seq.map (fun g -> {| Date = DateTime(g.Date.Year, g.Date.Month, 1); Count = g.Count |})
    |> Seq.sortBy (fun d -> d.Date)

let openCountByMonthDataPoints =
    openCountByMonth
    |> Seq.map (fun g -> {| Date = DateTime(g.Date.Year, g.Date.Month, 1); Count = g.Count |})
    |> Seq.sortBy (fun d -> d.Date)

let openSeries =
    Graph.Scattergl(
        name = "Created",
        x = (openDataPoints |> Seq.map (fun g -> g.Date)),
        y = (openDataPoints |> Seq.map (fun g -> g.Count)))

let closeSeries =
    Graph.Scattergl(
        name = "Closed",
        x = (closedDataPoints |> Seq.map (fun g -> g.Date)),
        y = (closedDataPoints |> Seq.map (fun g -> g.Count)))

let stillOpenSeries =
    Graph.Scattergl(
        name = "Open",
        x = (openCountByMonthDataPoints |> Seq.map (fun g -> g.Date)),
        y = (openCountByMonthDataPoints |> Seq.map (fun g -> g.Count)))

[openSeries; closeSeries; stillOpenSeries]
|> Chart.Plot
|> Chart.WithTitle("Issue report for the current year")

### How many times has dotnet/fsharp been forked?

In [13]:
let forks =
    async {
        return!
            gitHubClient.Repository.Forks.GetAll(organization, repositoryName)
            |> Async.AwaitTask
    } |> Async.RunSynchronously

In [14]:
let forksCreatedByMonth =
    forks
    |> Seq.groupBy (fun f -> DateTime(f.CreatedAt.Year, f.CreatedAt.Month,  f.CreatedAt.Day))
    |> Seq.map (fun (key, issues) -> {| Date = key; Count = issues.Count() |})
    |> Seq.sortBy (fun g -> g.Date)

let forksLastUpdateByMonth =
    forks
    |> Seq.groupBy (fun f -> DateTime(f.UpdatedAt.Year, f.UpdatedAt.Month,  f.UpdatedAt.Day))
    |> Seq.map (fun (key, issues) -> {| Date = key; Count = issues.Count() |})
    |> Seq.sortBy (fun g -> g.Date)

let forkCountByMonth =
    forksCreatedByMonth
    |> Seq.sortBy (fun g -> g.Date)
    |> Seq.map (fun g -> {| Date = g.Date; Count = g.Count |})

let forkCreationSeries =
    Graph.Scattergl(
        name = "created by month",
        x = (forksCreatedByMonth |> Seq.map (fun g -> g.Date)),
        y = (forksCreatedByMonth |> Seq.map (fun g -> g.Count)))

let forkTotalSeries =
    Graph.Scattergl(
        name = "running total",
        x = (forkCountByMonth |> Seq.map (fun g -> g.Date)),
        y = (forkCountByMonth |> Seq.map (fun g -> g.Count)))

let forkUpdateSeries =
    Graph.Scattergl(
        name = "last updated by month",
        x = (forksLastUpdateByMonth |> Seq.map (fun g -> g.Date)),
        y = (forksLastUpdateByMonth |> Seq.map (fun g -> g.Count)))

[forkCreationSeries; forkTotalSeries; forkUpdateSeries]
|> Chart.Plot
|> Chart.WithTitle("Fork activity")